### 셀 하나. 공휴일 다음날만 날짜 직접입력

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")

#########################################################
workday = "" # 빈문자열 or yyyy-mm-dd. 실효일. 특별히 원하는 날짜 없으면 빈문자열로 남겨두면 어제(평일)날짜로 작업(국경일 주의!)
cut_off_day = "240826" # yymmdd. 월말자산정리 기준일 
미배정건만 = False # 일단 배정건 미배정건 모두, 채권상태는 정상만
#########################################################

def get_previous_weekday(date):
    """날짜를 받아 이전 마지막 평일 날짜 yyyy-mm-dd return"""
    # 전일
    date = date - timedelta(days=1)
    # 전일부터 하루씩 차감하면서 평일이면 return
    while date.weekday() > 4:  # 월요일은 0이고, 금요일은 4
        date -= timedelta(days=1)
    return date.strftime('%Y-%m-%d')

def get_last_weekday_of_previous_month(workday:str):
    """'yyyy-mm-dd'를 받아 전달 마지막 평일 yymmdd return"""
    # 문자열을 날짜로
    workday = datetime.strptime(workday, "%Y-%m-%d") 
    
    # 이번 달의 첫 번째 날을 구한 후 하루를 빼서 지난달의 마지막 날을 구함
    first_day_of_this_month = workday.replace(day=1)
    last_day_of_previous_month = first_day_of_this_month - timedelta(days=1)
    
    # 마지막 날이 평일인지 확인하고, 평일이 아니면 하루씩 이전으로 이동
    while last_day_of_previous_month.weekday() > 4:  # 월요일은 0, 금요일은 4
        last_day_of_previous_month -= timedelta(days=1)
    
    return last_day_of_previous_month.strftime('%y%m%d') #yymmdd

def 파일명(폴더:str, 키워드:str, 부정키워드="") :
    file_list = os.listdir(폴더)
    if 부정키워드 == "" : 
        fn = [file for file in file_list if re.search(키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(키워드, file)!=None) & (re.search(부정키워드,file)==None)]
    if len(fn) == 1 :
        return join(폴더,fn[0])
    else :
        print("조건을 만족하는 파일이 둘 이상입니다.")
        

        
if workday == "" : 
    # 오늘 이전 중 가장 가까운 평일 날짜 가져오기
    workday = get_previous_weekday(datetime.today())
print(workday) 

# 업무대장에서 실효건 읽기
실효_ori = pd.read_excel(r"C:\Users\DATA\Desktop\0.new_신용회복 업무대장.xlsx", sheet_name = "실효", skiprows=3, dtype={"채무자키":str, "계좌키":str, "계좌조정일":str})
    # 빈행 삭제
실효_ori = 실효_ori.dropna(subset=["No."], axis=0)




# 계좌, 신복 기준데이터 읽기
if cut_off_day == "" : 
    cut_off_day = get_last_weekday_of_previous_month(workday)
    
dir_dataset = join(r"D:\3.자산\전산 dataset\솔림", cut_off_day, "pkl")
fn_account = 파일명(dir_dataset, "계좌조회새창_")
fn_credit = 파일명(dir_dataset, "신용회복_기준데이터")

account = pd.read_pickle(join(dir_dataset, fn_account))
credit = pd.read_pickle(join(dir_dataset, fn_credit))

#-------------------------------------------------------------------------------------------------------------------------------------------------------

# 복사본 만들기 및 정리하기                                                                        # 최초원금 = 현재원금
실효 = 실효_ori[["고객식별번호", "채무자키", "계좌키", "채무상태", "채권상태", "담당자", "최초원금", "계좌조정일", "채무구분", "전산 성명"]].copy()
    # 날짜 문자열로
실효["계좌조정일"] = 실효['계좌조정일'].apply(lambda x: x.split()[0])
    # 필터 : 날짜, 담당자(배정대기), 정상건 ############################################## 미확정
실효 = 실효.query('계좌조정일 == @workday')
if 미배정건만 : 실효 = 실효.query('담당자=="배정(대기)" and 채권상태=="정상"') 
else : 실효 = 실효.query('채권상태=="정상"') 
    
    # 열 추가
실효["확인일자"] = ""
실효["경과일"] = ""
실효["법원"] = ""
실효["채무상태"] = "신용회복(실효)"

print("병합전 건수 : ", len(실효))
# 계좌와 병합 : 채권구분, 매각사구분, OPB
실효 = pd.merge(실효, account[["계좌키", "채권구분", "매각사구분", "OPB"]], on='계좌키', how='left')
# 신용회복과 병합 : 접수번호
실효 = pd.merge(실효, credit[["계좌키","채무구분", "접수번호"]], on=['계좌키', '채무구분'], how='left')

print("병합수 건수 : ", len(실효))

# 열순서
실효 = 실효[["채무자키", "계좌키", "채무구분", "전산 성명", "계좌조정일", "확인일자", "담당자", "경과일", "법원", "접수번호", "채권구분", "매각사구분", "채무상태","OPB", "최초원금"]]
# 엑셀 저장
if len(실효) > 0 :
    실효.to_excel(join(r"C:\Users\DATA\Desktop", workday+"_일간신복실효.xlsx"), index=False)
    print(len(실효),"건 출력 완료")
    print("보증인 건은 차주 상태를 불러온 경우 오류날 수 있으니 재확인 하기바람")
else : 
    print("등록할 실효건이 없습니다.")

2024-11-01
병합전 건수 :  1
병합수 건수 :  1
1 건 출력 완료
보증인 건은 차주 상태를 불러온 경우 오류날 수 있으니 재확인 하기바람


### 결과확인

In [ ]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")


In [7]:
sheet_names = ["신청",]



신청 = pd.read_excel(r"C:\Users\DATA\Desktop\0.new_신용회복 업무대장.xlsx", sheet_name = "신청", skiprows=2, dtype={"채무자키":str, "계좌키":str, "신고기준일일":str})
unnamed_col_index = 신청.columns.get_loc(신청.filter(like="Unnamed").columns[0])
# 그 다음 열부터 데이터프레임 슬라이싱
신청 = 신청.iloc[:, unnamed_col_index+1:]